In [ ]:
import numpy as np
import random
np.random.seed(1234)

The functions are used to create random, but related, spline functions. A random spline is defined by multiplying the regression spline basis by a random coefficient. To create similarities across tasks, the coefficients are drawn from a mixture-of-gaussians.

In [ ]:
def random_functions(n_tasks, k_clusters, sigma_between, sigma_within):
    betas, zs = gaussian_mixture(n_tasks, k_clusters, sigma_between, sigma_within)
    functions = []
    for beta in betas:
        functions.append(spline_fun(beta))
        
    return functions, betas, zs

def gaussian_mixture(n_samples, k_clusters, sigma_between, sigma_within, dim=31):
    means = np.random.normal(0, sigma_between, (k_clusters, dim))
    cluster_props = np.random.dirichlet(k_clusters * [1.5])
    
    betas, zs = [], []
    for task in range(n_samples):
        z = np.random.choice(range(k_clusters), p = cluster_props)
        delta = np.random.normal(0, sigma_within, (1, dim))
        betas.append(means[z] + delta)
        zs.append(z)
        
    return np.vstack(betas), zs
        
def spline_fun(beta):
    def f(x):
        return np.matmul(basis(x), beta)
    return f
    
def basis(x, knots=None, degree = 3):
    if knots is None:
        knots = np.linspace(0, 1, 10)
        
    H = [np.ones(len(x))]
    for k in range(len(knots)):
        for j in range(1, degree + 1):
            H.append(pos(x - knots[k]) ** j)
    
    return np.vstack(H).T

def pos(x):
    x[x < 0] = 0
    return x

The block below uses these functions to create observations around these true random splines. `betas` are the spline coefficients for each task and `zs` are the cluster from which those coefficients are drawn.

In [ ]:
import pandas as pd

n_tasks = 15
f, betas, zs = random_functions(n_tasks, 6, 1, .2)
result = []

for i, fi in enumerate(f):
    x = np.random.uniform(0, 1, 100)
    result.append({
        "task": i,
        "x": x,
        "f": fi(x),
        "y": fi(x) + np.random.normal(0, .1, len(x))
    })

For later use, we write these simulated data to csv files. (The trick is to convert the dictionaries to data.frames, which are easy to write).

In [ ]:
result_df = pd.concat([pd.DataFrame(r) for r in result])
result_df.to_csv("~/Downloads/tasks.csv", index = False)

betas_df = np.hstack([np.arange(n_tasks)[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
betas_df = pd.DataFrame(betas_df)
betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
betas_df.to_csv("~/Downloads/betas.csv", index = False)

From R, you can use this code to plot the simulated data.

```
library(tidyverse)

betas <- read_csv("~/Downloads/betas.csv")
x <- read_csv("~/Downloads/tasks.csv")
ggplot(x %>% left_join(betas %>% select(task, cluster))) + 
    geom_point(aes(x, y, col = as.factor(cluster))) + 
    facet_wrap(~ task, scales = "free")
```